### Import data

In [1]:
include("importData.jl")
include("Hill.jl")
include("plot.jl")

# import G1, G2, and population data
conc_l, popl, g2l, g1l, g2_0l, g1_0l = setup_data("lapatinib");

┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, ::Colon, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = get_data(::String, ::String) at importData.jl:29
└ @ Main /home/farnazm/dde/lapatinib/DrugResponseModel.jl/src/importData.jl:29
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, ::Colon, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = get_data(::String, ::String) at importData.jl:37
└ @ Main /home/farnazm/dde/lapatinib/DrugResponseModel.jl/src/importData.jl:37


### Hill model, residual functions and optimization

In [ ]:
#  [EC50, b_steepness, alpha_min, alpha_max, beta_min, beta_max, tau1_mean, tau1_max, tau2_min, tau2_max, gamma1_max, gamma2_max]
# guess
guess = [[1.56, 0.05, 0.007, 0.008, 0.004, 0.05, 34.286, 6.56, 8.0, 7.0, 0.005, 0.035], 
[1.8, 0.02, 0.008, 0.008, 0.03, 0.0045, 20.0, 6.0, 20.0, 10.0, 0.003, 0.02],
[1.9, 0.03, 0.006, 0.007, 0.05, 0.0035, 15.0, 5.0, 30.0, 5.0, 0.002, 0.03],
[2.1, 0.04, 0.007, 0.005, 0.03, 0.005, 30.0, 3.0, 20.0, 4.0, 0.003, 0.02],
[1.8664, 0.097836, 0.00682578, 0.00958013, 0.00503788, 0.0433964, 38.941, 5.49712, 9.94827, 7.20807, 0.00294624, 0.0222821]]
# max num of steps
num_steps=1e4
best_fit = zeros(5)
params = zeros(12, 5)
for i in 1:5 
    best_fit[i], params[:, i] = optimize_hill(guess[i], conc_l, g1l, g2l, g1_0l, g2_0l, num_steps)
end


global optimization begins ...
Starting optimization with optimizer DiffEvoOpt{FitPopulation{Float64},RadiusLimitedSelector,BlackBoxOptim.AdaptiveDiffEvoRandBin{3},RandomBound{ContinuousRectSearchSpace}}
0.00 secs, 0 evals, 0 steps
50.03 secs, 415 evals, 325 steps, improv/step: 0.514 (last = 0.5138), fitness=867977.923149920
100.05 secs, 830 evals, 736 steps, improv/step: 0.457 (last = 0.4112), fitness=867880.120573017
150.15 secs, 1246 evals, 1153 steps, improv/step: 0.436 (last = 0.4005), fitness=867836.853779612
200.21 secs, 1661 evals, 1568 steps, improv/step: 0.437 (last = 0.4386), fitness=867819.658866004
250.26 secs, 2077 evals, 1984 steps, improv/step: 0.430 (last = 0.4038), fitness=867811.568835716
300.27 secs, 2492 evals, 2399 steps, improv/step: 0.413 (last = 0.3301), fitness=867811.077924479
350.29 secs, 2907 evals, 2814 steps, improv/step: 0.400 (last = 0.3277), fitness=867809.342142033
400.39 secs, 3322 evals, 3229 steps, improv/step: 0.385 (last = 0.2843), fitness=867803

In [ ]:
best, arg = findmin(best_fit)
ps = params[:,arg]

### Plot the data with the new set of parameters for DDE

In [ ]:
# easy way to convert the hill parameters to DDE parameters
p = getDDEparams(ps, conc_l)
# i showas the trial number, which could be from 1:control, ..., 8: maximum drug concentraation
p_1 = plotIt(p[:, 1], g1l, g2l, g2_0l, g1_0l, popl, 1, "Hill lapatinib")
p_2 = plotIt(p[:, 2], g1l, g2l, g2_0l, g1_0l, popl, 2, "")
p_3 = plotIt(p[:, 3], g1l, g2l, g2_0l, g1_0l, popl, 3, "")
p_4 = plotIt(p[:, 4], g1l, g2l, g2_0l, g1_0l, popl, 4, "")
p_5 = plotIt(p[:, 5], g1l, g2l, g2_0l, g1_0l, popl, 5, "")
p_6 = plotIt(p[:, 6], g1l, g2l, g2_0l, g1_0l, popl, 6, "")
p_7 = plotIt(p[:, 7], g1l, g2l, g2_0l, g1_0l, popl, 7, "")
p_8 = plotIt(p[:, 8], g1l, g2l, g2_0l, g1_0l, popl, 8, "")
plot(p_1, p_2, p_3, p_4, p_5, p_6, p_7, p_8)
plot!(size = (1000, 1000))

### Plot Hill curve for all of the DDE model parameters

In [ ]:
# plot
p__1 = plot(conc_l, p[1, :], label = "alpha", linewidth = 2, xlabel = "concentration [nM]", ylabel = "drug effect", line = (0.5, 4, :red))
p__2 = plot(conc_l, p[2, :], label = "beta", linewidth = 2, legend =:best, line = (0.5, 4, :red))
p__3 = plot(conc_l, p[3, :], label = "tau1", linewidth = 2, line = (0.5, 4, :red))
p__4 = plot(conc_l, p[4, :], label = "tau2", linewidth = 2, line = (0.5, 4, :red))
p__5 = plot(conc_l, p[5, :], label = "gamma1", linewidth = 2, line = (0.5, 4, :red))
p__6 = plot(conc_l p[6, :], label = "gamma2", linewidth = 2, line = (0.5, 4, :red))
plot(p__1, p__2, p__3, p__4, p__5, p__6)
plot!(size = (1000, 700))